# Connect Client

In [1]:
import weaviate 



client = weaviate.connect_to_wcs(
    cluster_url="https://dspy-cluster-8vxn8hp7.weaviate.network",  # Replace with your WCS URL
    auth_credentials=weaviate.auth.AuthApiKey("u4JlAXdnZnMcEfBUudwyG2kWW1AkinBunj6z"),
    skip_init_checks=True , # Replace with your WCS key
    headers={
        'X-Cohere-Api-Key': "J5iCZDAETMccnIgjnjYhRkvp3DlcWUueEjkdkZSS" # Replace with your Cohere API key
    }
)

# import weaviate
# import os
  
# # Set these environment variables
# URL = os.getenv("WCS_URL")
# APIKEY = os.getenv("WCS_API_KEY")
  
# # Connect to a WCS instance
# client = weaviate.connect_to_wcs(
#     cluster_url=URL,
#     auth_credentials=weaviate.auth.AuthApiKey(APIKEY))

# client = weaviate.Client(
#   url="https://dspy-cluster-8vxn8hp7.weaviate.network",
#   auth_client_secret=weaviate.auth.AuthApiKey("u4JlAXdnZnMcEfBUudwyG2kWW1AkinBunj6z"),
# )

# Create Schema

In [2]:
import weaviate.classes.config as wvcc

collection =client.collections.create(
    name="WeaviateBlogChunk",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_cohere
    (
        model="embed-multilingual-v3.0"
    ),
    properties=[
            wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
            wvcc.Property(name="author", data_type=wvcc.DataType.TEXT),
      ]
)

UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "WeaviateBlogChunk" already exists'}]}.

In [ ]:
import os
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    for folder_name in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, folder_name)
        if os.path.isdir(subfolder_path):
            index_file_path = os.path.join(subfolder_path, 'index.mdx')
            if os.path.isfile(index_file_path):
                with open(index_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    sentences = split_into_sentences(content)
                    sentence_chunks = chunk_list(sentences, 5)
                    sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
                    blog_chunks.extend(sentence_chunks)
    return blog_chunks

# Example usage
main_folder_path = './blog'
blog_chunks = read_and_chunk_index_files(main_folder_path)

In [ ]:
len(blog_chunks)

1182

In [3]:
print(blog_chunks[0])

NameError: name 'blog_chunks' is not defined

In [4]:
from weaviate.util import get_valid_uuid
from uuid import uuid4

blogs = client.collections.get("WeaviateBlogChunk")

for idx, blog_chunk in enumerate(blog_chunks):
    upload = blogs.data.insert(
        properties={
            "content": blog_chunk
        }
    )

NameError: name 'blog_chunks' is not defined

In [5]:
client.cluster

client.schema.get()

client.collections.get("WeaviateBlogChunk")

In [7]:
first = client.collections.get("WeaviateBlogChunk")
response = first.query.fetch_objects()


Now, you can expose Prometheus-compatible metrics for monitoring. Combine this with a standard Prometheus/Grafana setup to create visual dashboards for metrics around latencies, import speed, time spent on vector vs object storage, memory usage, and more. ![Importing Data into Weaviate](./img/weaviate-sample-dashboard-importing.png "Importing Data Into Weaviate")

### Example
In a hypothetical scenario, you might be importing a large dataset. At one point the import process might slow down. You could then check your dashboards, where you might see that the vector indexing process is still running fast, while the object indexing slowed down.
He developed an Autonomous Testing Agent to enhance software testing efficiency, harnessing the power of [SuperAGI](https://www.linkedin.com/company/superagi/) and Weaviate. Meanwhile, [BYTE](https://lablab.ai/event/cohere-coral-hackathon/byte/byte-ai-based-nutrition-app), an AI-based nutrition app, clinched the top spot at the [Coral Cohere Hackath